In [111]:
grid = [[character for character in row.strip()] for row in open("sample.txt", "r").readlines()]
grid

[['O', 'O', 'O', 'O', 'O'],
 ['O', 'X', 'O', 'X', 'O'],
 ['O', 'O', 'O', 'O', 'O'],
 ['O', 'X', 'O', 'X', 'O'],
 ['O', 'O', 'O', 'O', 'O']]

In [1]:
def in_bounds(grid, row, col):
    max_row = len(grid)
    max_col = len(grid[0])

    if (max_row > row >= 0) and (max_col > col >= 0):
        return True
    else:
        return False

def adjacent(grid, row, col):
    near = [(row-1, col), (row, col + 1), (row+1, col), (row, col-1)]
    return [(row, col)for row, col in near if in_bounds(grid, row, col)]
 

First semi-working solution:
Overcounts single split regions like:

OOOOO  
OXOXO  
OOOOO  
OXOXO  
OOOOO

where X get the correct perimeter of 16, but then overcounts it 4 times.

In [ ]:
from collections import defaultdict
grid = [[character for character in row.strip()] for row in open("sample.txt", "r").readlines()]

grid_map = defaultdict(list) 

for row, _ in enumerate(grid):
    for col, _ in enumerate(grid[row]):
        open_to = 4 
        character = grid[row][col]

        for near_row, near_col in adjacent(grid, row, col):
            if grid[near_row][near_col] == character:
                open_to -= 1
        grid_map[character].append(open_to)


total = 0
for region in list(grid_map.values()):
    total += sum(region)*len(region)
total      

820

In [112]:
grid_map

defaultdict(list,
            {'O': [2,
              2,
              1,
              2,
              2,
              2,
              2,
              2,
              1,
              2,
              0,
              2,
              1,
              2,
              2,
              2,
              2,
              2,
              1,
              2,
              2],
             'X': [4, 4, 4, 4]})

In [109]:
for perimeter in list(grid_map.values()):
    print(sum(perimeter))

36
16


In [110]:
for area in list(grid_map.values()):
    print(len(area))

21
4


## Second try:
works for single small regions, but not larger regions of the same character :( 

In [8]:
from collections import defaultdict
grid = [[character for character in row.strip()] for row in open("sample.txt", "r").readlines()]

grid_map = defaultdict(list) 
i = 0
for row, _ in enumerate(grid):
    for col, _ in enumerate(grid[row]):
        perimeter = 4 
        character = grid[row][col]

        for near_row, near_col in adjacent(grid, row, col):
            if grid[near_row][near_col] == character:
                perimeter -= 1
        if perimeter == 4:
            grid_map[f"{character}+{i}"].append(perimeter)
            i += 1
        else:
            grid_map[character].append(perimeter)


total = 0
for region in list(grid_map.values()):
    total += sum(region)*len(region)
total      

2130

In [12]:
for key, value in grid_map.items():
    print(f"{key} =  {len(value)} * {sum(value)} = {len(value)*sum(value)}")

R =  12 * 18 = 216
I =  18 * 30 = 540
C =  14 * 28 = 392
F =  10 * 18 = 180
V =  13 * 20 = 260
J =  11 * 20 = 220
C+0 =  1 * 4 = 4
E =  13 * 18 = 234
M =  5 * 12 = 60
S =  3 * 8 = 24


# Flood fill
Got hint from HyperNeutrino's youtube to look into Flood fill, to count every single region.

In [ ]:
def in_bounds(grid, row, col):
    return 0 <= row < len(grid) and 0 <= col < len(grid[0])

def adjacent(row, col):
    return [(row-1, col), (row, col + 1), (row+1, col), (row, col-1)]
    
def dfs_flood_fill(grid, row: int, col: int, visited=None) -> list:
    if visited is None:
        visited = set()
    
    if not in_bounds(grid, row, col) or (row, col) in visited:
        return []
    
    visited.add((row, col))
    start = grid[row][col]

    lst = [(row, col)]

    for near_row, near_col in adjacent(row, col):
        if in_bounds(grid, near_row, near_col) and grid[near_row][near_col] == start:
            lst.extend(dfs_flood_fill(grid, near_row, near_col, visited))

    return lst

def perimeter_calc(region: list[tuple]) -> int:
    total_perimeter = 0
    for row, col in region:
        perimeter = 4
        for near_row, near_col in adjacent(row, col):
            if  (near_col, near_row) in region:
                perimeter -= 1
        total_perimeter += perimeter
        
    return total_perimeter*len(region)

grid = [[character for character in row.strip()] for row in open("sample.txt", "r").readlines()]

visited = set()
regions = []
for row in range(len(grid)):
    for col in range(len(grid[row])):
        if (row, col) in visited:
            continue
        region = dfs_flood_fill(grid, row, col)
        visited |= set(region)
        regions.append(region)

sum(map(perimeter_calc, regions))    